# Prediction and Power Bi Input.
<a id='top_cell'></a>
Following the model training on (Forecasting Model and Training.ipynb), we make a prediction on each experiment an generate an Excel file for the Power BI dashboard data on Predict

This Notebook was originally on Colab.


In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error
import seaborn as sns
import pandas as pd
from sklearn.utils import check_array
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
import math
import statistics
from datetime import time
from keras.regularizers import l2
import tensorflow as tf
from pickle import dump, load
from statistics import mean
from __future__ import print_function
import os
import copy
import warnings
import graphviz
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
import multiprocessing
import pickle
import sklearn
from datetime import time
from datetime import datetime
# Advanced plotting functionality with seaborn
import seaborn as sns
sns.set(style="whitegrid")  # can set style depending on how you'd like it to look

##Dataframes
First we upload the dataframes generated, on last notebook before training, and we make a few changes for its prediction and further adjust to the dataframe that is going to be saved.

In [87]:
#df=pd.read_csv('/content/drive/MyDrive/DS4A/dataframes/Data_Predict_Hurtos_Barrios.csv')
#df=pd.read_csv('/content/drive/MyDrive/DS4A/dataframes/Data_Predict_Hurtos_Comunas.csv')
#df=pd.read_csv('/content/drive/MyDrive/DS4A/dataframes/Data_Predict_Captura_Barrios.csv')
df=pd.read_csv('/content/drive/MyDrive/DS4A/dataframes/Data_Predict_Captura_Comunas.csv')
df.head()

,Unnamed: 0,Comuna 1,Comuna 10,Comuna 11,Comuna 12,Comuna 13,Comuna 14,Comuna 15,Comuna 16,Comuna 2,...,Comuna 6,Comuna 7,Comuna 8,Comuna 9,Corregimiento 50,Corregimiento 60,Corregimiento 70,Corregimiento 80,Corregimiento 90,Fecha
0,0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-01-01
1,1,0.0,3.0,0.0,3.0,0.0,0.0,2.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-01-02
2,2,0.0,4.0,2.0,2.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,2015-01-03
3,3,3.0,5.0,1.0,0.0,0.0,1.0,0.0,2.0,1.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,2015-01-04
4,4,1.0,8.0,2.0,0.0,0.0,2.0,0.0,1.0,1.0,...,0.0,0.0,2.0,1.0,0.0,0.0,2.0,0.0,0.0,2015-01-05


It is posible to have missing days beacuse if one day there was not theft or capture reported it will miss but we need the all dates to a proper predict so we resample day and add missing values with 0

In [88]:
df['Fecha'] = pd.to_datetime(df['Fecha'])
df = df.set_index('Fecha').resample('1d').asfreq().reset_index().replace(np.nan, 0)

In [89]:
df.head()

,Fecha,Unnamed: 0,Comuna 1,Comuna 10,Comuna 11,Comuna 12,Comuna 13,Comuna 14,Comuna 15,Comuna 16,...,Comuna 5,Comuna 6,Comuna 7,Comuna 8,Comuna 9,Corregimiento 50,Corregimiento 60,Corregimiento 70,Corregimiento 80,Corregimiento 90
0,2015-01-01,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2015-01-02,1.0,0.0,3.0,0.0,3.0,0.0,0.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2015-01-03,2.0,0.0,4.0,2.0,2.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0
3,2015-01-04,3.0,3.0,5.0,1.0,0.0,0.0,1.0,0.0,2.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
4,2015-01-05,4.0,1.0,8.0,2.0,0.0,0.0,2.0,0.0,1.0,...,1.0,0.0,0.0,2.0,1.0,0.0,0.0,2.0,0.0,0.0


We save the dates of the Y test on an array for the further match with each prediction

In [90]:
Fechas = df.iloc[1:,0]
Fechas = Fechas[int(round(len(Fechas)*0.8, 0)):]

We drop the noise columns

In [93]:
df=df.drop(columns=['Fecha','Unnamed: 0'])

We make the same preprocess as before for the data to have a correct input to the model

In [94]:
#Scale data between 0,1 for LSTM/GRU
MMS = MinMaxScaler(feature_range=(0, 1))
df_scale = pd.DataFrame(MMS.fit_transform(df.values), columns=df.columns)


#  Remember to shift the data so we are predicting one time step future
X = df_scale.iloc[:-1:]
#y = df_scale.iloc[1:,:-49] #locationIDs Barrios Hurtos
y = df_scale.iloc[1:,:] #locationIDs Comunas Hurtos y Captura
#y = df_scale.iloc[1:,:-32] #locationIDs Barrios Captura

# Train-Test-Split-90/10
X_train, X_test = X[:int(round(len(X)*0.8, 0))], X[int(round(len(X)*0.8, 0)):]
y_train, y_test = y[:int(round(len(y)*0.8, 0))], y[int(round(len(y)*0.8, 0)):]

#Reshape for LSTM/GRU layers
Xtr = X_train.values.reshape((1, X_train.shape[0], X_train.shape[1]))
#Xtr = X_train.reshape((X_train.shape[0],look_back, X_train.shape[2]))
ytr = y_train.values.reshape((1,y_train.shape[0], y_train.shape[1]))
Xte = X_test.values.reshape((1, X_test.shape[0], X_test.shape[1]))
#Xte = X_test.reshape(( X_test.shape[0],look_back, X_test.shape[2]))
yte = y_test.values.reshape((1,y_test.shape[0], y_test.shape[1]))

#Check data validity
print(len(X_train) + len(X_test), len(y_train) + len(y_test))

2464 2464


##Model load and initialization
As the last notebook we need to make the model but instead of training again we load the pre saved weights and generate the prediction

First we initialize the metric used

In [97]:
import keras.backend as K
#K._BACKEND == 'tensorflow'

def custom_r2(y_true, y_pred):
    baseline = K.sum((y_true - K.mean(y_true))**2)
    model_fit = K.sum((y_true - y_pred)**2)
    return 1. - model_fit/baseline

In [98]:
from keras.models import Sequential, Model, load_model
from keras.layers import LSTM, Dense, Dropout, GRU, TimeDistributed, Input, merge, Activation
from keras.layers import Reshape
from tensorflow.keras.optimizers import Adamax

We make the model Function and Initialize it

In [99]:
def build_model_2(n_features, dropout, n_outputs, compile_model=1, load_weights=0, loss='mse', lr=0.01):
    '''
    Builds a n-Layer neural_network 
    
    Parmaters
    ---------
    
    '''
    model = Sequential()

    #DENSE - INPUT
    model.add(Conv1D(filters=2048, kernel_size=2, activation='relu', input_shape=(None, n_features),padding="same"))
    model.add(Dropout(dropout))
    model.add(GRU(4096, return_sequences=True))
    model.add(Dropout(dropout))
    model.add(TimeDistributed(Dense(1000, activation='relu')))
    model.add(TimeDistributed(Dense(n_outputs, activation='relu')))

    if compile_model == 1:
        model.compile(loss=loss, optimizer=Adamax(lr=lr), metrics=[custom_r2])

    if load_weights == 1:
        model.load_weights('/content/drive/MyDrive/DS4A/logs/'+file_name+'.h5')
        
    return model

In [100]:
#1 - Build Model
model = build_model_2(n_features=Xtr.shape[2], dropout=0.1, n_outputs=ytr.shape[2], compile_model=1, load_weights=0, lr=0.0001, loss = 'msle')
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_5 (Conv1D)           (None, None, 2048)        88064     
                                                                 
 dropout_10 (Dropout)        (None, None, 2048)        0         
                                                                 
 gru_5 (GRU)                 (None, None, 4096)        75522048  
                                                                 
 dropout_11 (Dropout)        (None, None, 4096)        0         
                                                                 
 time_distributed_10 (TimeDi  (None, None, 1000)       4097000   
 stributed)                                                      
                                                                 
 time_distributed_11 (TimeDi  (None, None, 21)         21021     
 stributed)                                           

We load the Weights depending on the dataframe used

In [101]:
#model.load_weights("/content/drive/MyDrive/DS4A/logs/Model_1_Medellin_BarriosSinFecha_Hurto_2015.h5")
#model.load_weights("/content/drive/MyDrive/DS4A/logs/Model_1_Medellin_ComunaSinFecha_Hurto_2015.h5")
#model.load_weights("/content/drive/MyDrive/DS4A/logs/Model_1_Medellin_BarrioSinFecha_Captura_2015.h5")
model.load_weights("/content/drive/MyDrive/DS4A/logs/Model_1_Medellin_ComunaSinFecha_Captura_2015.h5")

## Prediction and Dataframe Generation

###Prediction
First we make a prediction on Train and Test sets leaving the test set at the end beacuse it will be the one that will be save to feed Power BI dashboard

In [104]:
pred = model.predict(Xtr)

In [105]:
baseline = np.sum((ytr - np.mean(ytr))**2)
model_fit = np.sum((ytr - pred)**2)
print(1. - model_fit/baseline)

0.9979918399347593


In [102]:
pred = model.predict(Xte)

In [103]:
baseline = np.sum((yte - np.mean(yte))**2)
model_fit = np.sum((yte - pred)**2)
print(1. - model_fit/baseline)

0.984347708387635


### Initial dataframes of Real and Predict values
We generate two dataframes one with the real values and one with the Predicted that contained the dates on a column and the rest of columns with the location names in the same order of original dataframe load

In [272]:
Original = df_scale.iloc[1:,:] [int(round(len(df_scale)*0.8, 0))-1:]

In [273]:
Ori_data_S = MMS.inverse_transform(Original.values)
Original_S = pd.DataFrame(Ori_data_S[:,:],columns=Original.columns[:])
Original_S.head()

,Comuna 1,Comuna 10,Comuna 11,Comuna 12,Comuna 13,Comuna 14,Comuna 15,Comuna 16,Comuna 2,Comuna 3,...,Comuna 5,Comuna 6,Comuna 7,Comuna 8,Comuna 9,Corregimiento 50,Corregimiento 60,Corregimiento 70,Corregimiento 80,Corregimiento 90
0,1.0,2.0,2.0,1.0,1.0,4.0,0.0,0.0,1.0,9.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,3.0,8.0,0.0,6.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,4.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,4.0,2.0,3.0,3.0,2.0,0.0,0.0,3.0,5.0,...,1.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,7.0,0.0,1.0,5.0,0.0,0.0,1.0,1.0,1.0,...,2.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [276]:
Original_S['Fecha'] = Fechas.values

In [277]:
Original_S.head()

,Comuna 1,Comuna 10,Comuna 11,Comuna 12,Comuna 13,Comuna 14,Comuna 15,Comuna 16,Comuna 2,Comuna 3,...,Comuna 6,Comuna 7,Comuna 8,Comuna 9,Corregimiento 50,Corregimiento 60,Corregimiento 70,Corregimiento 80,Corregimiento 90,Fecha
0,1.0,2.0,2.0,1.0,1.0,4.0,0.0,0.0,1.0,9.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-05-26
1,0.0,3.0,8.0,0.0,6.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-05-27
2,0.0,1.0,4.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2020-05-28
3,0.0,4.0,2.0,3.0,3.0,2.0,0.0,0.0,3.0,5.0,...,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,2020-05-29
4,1.0,7.0,0.0,1.0,5.0,0.0,0.0,1.0,1.0,1.0,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-05-30


In [278]:
Predict = pred.reshape(pred.shape[1:])

In [280]:
Pred_data_S = MMS.inverse_transform(Predict)
Predict_S = pd.DataFrame(Pred_data_S,columns=Original.columns)
Predict_S.head()

,Comuna 1,Comuna 10,Comuna 11,Comuna 12,Comuna 13,Comuna 14,Comuna 15,Comuna 16,Comuna 2,Comuna 3,...,Comuna 5,Comuna 6,Comuna 7,Comuna 8,Comuna 9,Corregimiento 50,Corregimiento 60,Corregimiento 70,Corregimiento 80,Corregimiento 90
0,1.080853,0.000000,1.165934,0.095874,0.493401,3.059544,0.000000,0.000000,0.000000,6.914467,...,0.187015,0.220647,0.229546,1.588782,0.132251,0.026054,0.193958,0.0,0.085221,0.017522
1,0.491644,2.136284,7.058884,0.000000,5.018618,0.408375,0.000000,0.000000,1.365605,0.532734,...,0.127243,0.009482,0.174694,0.129068,0.081694,0.034266,0.039230,0.0,0.119354,0.000000
2,0.233875,1.021949,3.755360,0.810022,2.799063,1.032274,0.000000,0.000000,0.000000,0.263722,...,0.210078,0.043071,1.026983,0.038561,0.130044,0.000000,0.149308,0.0,0.806077,0.010317
3,0.200185,4.092966,1.826188,2.973738,2.890524,1.984437,0.025577,0.000000,2.970175,5.077706,...,1.075150,0.000000,0.000000,1.904142,1.117514,0.000000,0.000000,0.0,0.000000,0.000000
4,1.143832,6.960531,0.000000,1.025418,5.024642,0.116633,0.000000,0.908971,1.044429,0.939945,...,1.965353,1.895469,0.005222,0.899867,0.031199,0.000000,0.000000,0.0,0.000000,0.000000


In [282]:
Predict_S['Fecha'] = Fechas.values

In [283]:
Predict_S.head()

,Comuna 1,Comuna 10,Comuna 11,Comuna 12,Comuna 13,Comuna 14,Comuna 15,Comuna 16,Comuna 2,Comuna 3,...,Comuna 6,Comuna 7,Comuna 8,Comuna 9,Corregimiento 50,Corregimiento 60,Corregimiento 70,Corregimiento 80,Corregimiento 90,Fecha
0,1.080853,0.000000,1.165934,0.095874,0.493401,3.059544,0.000000,0.000000,0.000000,6.914467,...,0.220647,0.229546,1.588782,0.132251,0.026054,0.193958,0.0,0.085221,0.017522,2020-05-26
1,0.491644,2.136284,7.058884,0.000000,5.018618,0.408375,0.000000,0.000000,1.365605,0.532734,...,0.009482,0.174694,0.129068,0.081694,0.034266,0.039230,0.0,0.119354,0.000000,2020-05-27
2,0.233875,1.021949,3.755360,0.810022,2.799063,1.032274,0.000000,0.000000,0.000000,0.263722,...,0.043071,1.026983,0.038561,0.130044,0.000000,0.149308,0.0,0.806077,0.010317,2020-05-28
3,0.200185,4.092966,1.826188,2.973738,2.890524,1.984437,0.025577,0.000000,2.970175,5.077706,...,0.000000,0.000000,1.904142,1.117514,0.000000,0.000000,0.0,0.000000,0.000000,2020-05-29
4,1.143832,6.960531,0.000000,1.025418,5.024642,0.116633,0.000000,0.908971,1.044429,0.939945,...,1.895469,0.005222,0.899867,0.031199,0.000000,0.000000,0.0,0.000000,0.000000,2020-05-30


### Power BI dataframe
Finally for the power bi dataframe we pivot Real and Predict dataframes in order to have the information only in 3 columns, then we concatenate the pivoted datframes and save the result

In [284]:
df_unpivot = pd.melt(Predict_S, id_vars='Fecha', value_vars=Predict_S.columns[:-1])
df_unpivot.head()

,Fecha,variable,value
0,2020-05-26,Comuna 1,1.080853
1,2020-05-27,Comuna 1,0.491644
2,2020-05-28,Comuna 1,0.233875
3,2020-05-29,Comuna 1,0.200185
4,2020-05-30,Comuna 1,1.143832


In [285]:
df_unpivot_2 = pd.melt(Original_S, id_vars='Fecha', value_vars=Original_S.columns[:-1])
df_unpivot_2.head()

,Fecha,variable,value
0,2020-05-26,Comuna 1,1.0
1,2020-05-27,Comuna 1,0.0
2,2020-05-28,Comuna 1,0.0
3,2020-05-29,Comuna 1,0.0
4,2020-05-30,Comuna 1,1.0


In [288]:
#df_unpivot.rename(columns = {'value':'Pred','variable':'Barrio'}, inplace = True)
df_unpivot.rename(columns = {'value':'Pred','variable':'Comuna'}, inplace = True)

In [289]:
df_unpivot.head()

,Fecha,Comuna,Pred
0,2020-05-26,Comuna 1,1.080853
1,2020-05-27,Comuna 1,0.491644
2,2020-05-28,Comuna 1,0.233875
3,2020-05-29,Comuna 1,0.200185
4,2020-05-30,Comuna 1,1.143832


In [290]:
df_unpivot['Real'] = df_unpivot_2['value']

In [291]:
df_unpivot.tail(50)

,Fecha,Comuna,Pred,Real
10303,2021-08-12,Corregimiento 90,0.000000,0.0
10304,2021-08-13,Corregimiento 90,0.000000,0.0
10305,2021-08-14,Corregimiento 90,0.000000,0.0
10306,2021-08-15,Corregimiento 90,0.000000,0.0
10307,2021-08-16,Corregimiento 90,0.000000,0.0
10308,2021-08-17,Corregimiento 90,0.000000,0.0
10309,2021-08-18,Corregimiento 90,0.000000,0.0
10310,2021-08-19,Corregimiento 90,0.000000,0.0
10311,2021-08-20,Corregimiento 90,0.000000,0.0
10312,2021-08-21,Corregimiento 90,0.000000,0.0


In [292]:
#df_unpivot.to_excel("/content/drive/MyDrive/DS4A/dataframes/Barrios_Pronostico_Hurtos.xlsx")
#df_unpivot.to_excel("/content/drive/MyDrive/DS4A/dataframes/Comunas_Pronostico_Hurtos.xlsx")
#df_unpivot.to_excel("/content/drive/MyDrive/DS4A/dataframes/Barrios_Pronostico_Captura.xlsx")
df_unpivot.to_excel("/content/drive/MyDrive/DS4A/dataframes/Comunas_Pronostico_Captura.xlsx")